In [34]:
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import urllib.request as libreq

In [48]:
soup = BeautifulSoup(open(r'classification_codes_data\arxiv_category_taxonomy.html'), 'html.parser')
classification_codes = soup.get_text().split('\n')

In [27]:
main_cls = ['Computer Science', 'Economics', 'Electrical Engineering and Systems Science', 'Mathematics', 'Physics', 'Quantitative Biology', 'Quantitative Finance', 'Statistics']

In [50]:
# classification_codes = [code for code in classification_codes if (code != '' and code not in main_cls)]
classification_codes = [code for code in classification_codes if (code != '')]

In [51]:
classification_codes

['Computer Science',
 'cs.AI (Artificial Intelligence)',
 'Covers all areas of AI except Vision, Robotics, Machine Learning, Multiagent Systems, and Computation and Language (Natural Language Processing), which have separate subject areas. In particular, includes Expert Systems, Theorem Proving (although this may overlap with Logic in Computer Science), Knowledge Representation, Planning, and Uncertainty in AI. Roughly includes material in ACM Subject Classes I.2.0, I.2.1, I.2.3, I.2.4, I.2.8, and I.2.11.',
 'cs.AR (Hardware Architecture)',
 'Covers systems organization and hardware architecture. Roughly includes material in ACM Subject Classes C.0, C.1, and C.5.',
 'cs.CC (Computational Complexity)',
 'Covers models of computation, complexity classes, structural complexity, complexity tradeoffs, upper and lower bounds. Roughly includes material in ACM Subject Classes F.1 (computation by abstract devices), F.2.3 (tradeoffs among complexity measures), and F.4.3 (formal languages), altho

In [19]:
with(open('arxiv_classification.csv', 'r')) as f:
    classification_codes = f.readlines()
    print(len(classification_codes))



317
